In [2]:
import torch
import time 
import copy
import os
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
import pickle

In [4]:
train_dataset = torchvision.datasets.ImageFolder('dataset/train')
train_class_names = train_dataset.classes

test_dataset = torchvision.datasets.ImageFolder('dataset/test')
test_class_names = train_dataset.classes

print(train_class_names, test_class_names)

['glasses', 'no_glasses'] ['glasses', 'no_glasses']


In [5]:
batch_size = 32
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_transform = torchvision.transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transfrom = torchvision.transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder('dataset/train', train_transform)

test_dataset = torchvision.datasets.ImageFolder('dataset/test', train_transform) 

trainloader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

valloader = torch.utils.data.DataLoader(test_dataset, 
                                        batch_size=batch_size,
                                        shuffle=False)


In [6]:
from copy import deepcopy
class Main_train_loop():
    def __init__(self, batch_size, trainloader, testloader, Device, model): 
        self.batch_size = batch_size
        self.trainloader = trainloader
        self.testloader = testloader
        self.device = Device
        self.model = model
        self.loss_arr = []
        self.loss_arr_test = []
        self.metric_train = []
        self.metric_test = []
        # self.get_part_train_loop()
        
    def get_part_train_loop(self): 
        self.loss_fn = nn.CrossEntropyLoss()
        self.opt = optim.Adam(self.model.parameters(), lr=0.001)

    def accuracy(self,y_predicts,data_y):
        self.y_predicts = y_predicts
        self.data_y = data_y
        acc = (y_predicts == data_y).sum().item() / y_predicts.size(0)
        return acc

    def train_nn(self,trainloader, model, opt, loss_fn, device):
        running_loss = []
        model.train()
        acc_list = []
        for i, data in tqdm(enumerate(trainloader, 0)):
            inputs, labels = data 
            inputs = inputs.to(device)
            labels = labels.to(device)

            opt.zero_grad() 
            outputs = model(inputs)
            y_pred = outputs.argmax(dim=1) 
            loss = loss_fn(outputs, labels)
            loss.backward() 
            opt.step() 

            running_loss.append(loss.item())
            acc_list.append(self.accuracy(y_pred,
                                           labels))

        return model, running_loss,acc_list


    def eval_nn(self,testloader, model, loss_fn, device):
        running_loss_test = []
        acc_list = []

        model.eval()
        predicts = np.array([])
        
        for i, data in enumerate(testloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)

            outputs = model(inputs)
            y_pred = outputs.cpu().argmax(dim=1)
            predicts = np.append(predicts,y_pred)
            loss = loss_fn(outputs.cpu(), labels)

            running_loss_test.append(loss.item())
            acc_list.append(self.accuracy(y_pred,
                                           labels))
        return running_loss_test, predicts.astype(int),acc_list

    def training_loop(self, max_epochs=20):
      
        for epoch in range(max_epochs):
            start_time = time.time()
            self.model, running_loss,acc_train = self.train_nn(self.trainloader, 
                                                             self.model,
                                         self.opt, self.loss_fn, self.device)
            running_loss_test, _,acc_eval = self.eval_nn(self.testloader,
                                         self.model,
                                         self.loss_fn, self.device)

            self.loss_arr.append(np.mean(running_loss))
            self.loss_arr_test.append(np.mean(running_loss_test))


            self.metric_train.append(np.mean(acc_train))
            self.metric_test.append(np.mean(acc_eval))
            if np.mean(acc_eval)>=np.max(self.metric_test):
                self.best_model = deepcopy(self.model)
                self.best_epoch = epoch

            
            duration = time.time() - start_time
            print(f"""EPOCH {epoch} : train_loss: {self.metric_train[-1]:.5f} 
            test_loss: {self.metric_test[-1]:.5f}.
            Заняло времени {round(duration,2)} секунд""")


In [7]:
mobilenetv3 = torchvision.models.mobilenet_v3_small(pretrained=True)
mobilenetv3.fc = nn.Linear(320, 2)
mobilenetv3.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to C:\Users\Asus/.cache\torch\hub\checkpoints\mobilenet_v3_small-047dcff4.pth


MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride

In [8]:
main_loop = Main_train_loop(batch_size,trainloader,valloader,device,mobilenetv3)
main_loop.get_part_train_loop()
main_loop.training_loop(20)

106it [05:50,  3.31s/it]
0it [00:00, ?it/s]

EPOCH 0 : train_loss: 0.98113 
            test_loss: 0.99929.
            Заняло времени 427.44 секунд


1it [00:06,  6.30s/it]


KeyboardInterrupt: 

In [ ]:
# torch.save(main_loop.best_model, 'mobilev3_model.pt')